# Before We Start
1. Python, Pandas, VS Code setup instructions 
2. Dataset link or simulated pose data provided 

# Step 0: Install & Import Modules

In [ ]:
# Install stuff

In [3]:
# Import stuff
import pandas as pd
from pandas import DataFrame
import copy

# Step 1: Load and examine pose data 

In [4]:
raw_pose_data = [
    {"people": [{"pose_keypoints_2d": [378.62797810302845, 108.21663023365875, 1.0, 
                                       378.6279781030284, 179.87733615852062, 1.0, 
                                       316.19092739621794, 177.74880033897023, 1.0, 
                                       214.0212080578009, 187.68196749687195, 1.0, 
                                       116.10856035848451, 191.93903913597262, 1.0, 
                                       441.06502880983885, 182.005871978071, 1.0, 
                                       544.653772027956, 190.52001525627236, 1.0, 
                                       641.1473958475722, 190.52001525627236, 1.0, 
                                       344.57140499022273, 367.89800021880205, 1.0, 
                                       353.0855482684242, 533.9237941437297, 1.0, 
                                       365.8567631857263, 695.6925164295568, 1.0, 
                                       419.77967061433526, 369.3170240985023, 1.0, 
                                       411.2655273361338, 532.5047702640296, 1.0, 
                                       389.9801691406303, 690.0164209107558, 1.0, 
                                       367.27578706542647, 95.4454153163565, 1.0, 
                                       392.81821690003085, 95.4454153163565, 1.0, 
                                       351.66652438872393, 101.12151083515744, 1.0, 
                                       414.1035750955343, 102.5405347148577, 1.0]}], 
                "canvas_height": 768, 
                "canvas_width": 768},
                ]
pose2 = [{"people": [{"pose_keypoints_2d": [384.20129182603614, 108.05394766065803, 1.0, 388.2368462880452, 191.64757580227325, 1.0, 321.93845155504005, 191.07106802198626, 1.0, 249.29847123887805, 266.0170794592963, 1.0, 176.65849092271594, 328.2799197302924, 1.0, 454.5352410210503, 192.22408358256024, 1.0, 521.4101435343424, 260.25200165642633, 1.0, 602.1212327745226, 331.73896641201446, 1.0, 344.99876276652026, 375.5535577138265, 1.0, 353.0698716905381, 534.6697050730388, 1.0, 346.15177832709423, 668.4195100996228, 1.0, 430.32191424899634, 373.2475265926785, 1.0, 425.7098520067002, 531.2106583913167, 1.0, 409.5676341586643, 685.7147435082329, 1.0, 371.5181206597222, 95.37077649434406, 1.0, 396.8844629923502, 95.37077649434406, 1.0, 357.6819339328342, 105.74791653951007, 1.0, 417.63874308268225, 105.74791653951007, 1.0]}], "canvas_height": 768, "canvas_width": 768}]

def extract_keypoints(data):
    '''
    Convert the raw pose JSON into a dict of coordinate pairs
    '''
    payload = data[0]["people"][0]["pose_keypoints_2d"]
    payload = list(filter((1.0).__ne__,payload))
    return payload

points = extract_keypoints(raw_pose_data)

def generate_column_labels(coords=18):
    '''
    Make an array with the names of all of the columns instead of handcoding it.
    '''
    labels = []
    for i in range(coords):
        formatted_number = f"{i:02d}"
        labels.append(formatted_number + "x")
        labels.append(formatted_number + "y")
    return labels

labels = generate_column_labels()

coord_to_name = ["head",            # Using the Pose Output Format (COCO), which appears to be what OpenPoses uses
                 "hips",
                 "left_shoulder",
                 "left_elbow",
                 "left_hand",
                 "right_shoulder",
                 "right_elbow",
                 "right_hand",
                 "left_hip",
                 "left_knee",
                 "left_foot",
                 "right_hip",
                 "right_knee",
                 "right_foot",
                 "left_eye",
                 "right_eye",
                 "left_ear",
                 "right_ear"]

def prep_dataframe(labels):
    '''
    Make a DF with the proper labels
    '''
    return DataFrame(columns=labels)

def store_keypoints(points, labels, df):
    '''
    Store pose coords within a DF. 
    '''
    if len(points) != len(labels):
        print("Error: Size mismatch between coordinates list and labels")
        return df
    temp_points = copy.copy(points)
    df.loc[len(df)] = temp_points

df = prep_dataframe(labels)
store_keypoints(points,labels,df)
points = extract_keypoints(pose2)
store_keypoints(points,labels,df)
print(df)

          00x         00y         01x         01y         02x         02y  \
0  378.627978  108.216630  378.627978  179.877336  316.190927  177.748800   
1  384.201292  108.053948  388.236846  191.647576  321.938452  191.071068   

          03x         03y         04x         04y  ...         13x  \
0  214.021208  187.681967  116.108560  191.939039  ...  389.980169   
1  249.298471  266.017079  176.658491  328.279920  ...  409.567634   

          13y         14x        14y         15x        15y         16x  \
0  690.016421  367.275787  95.445415  392.818217  95.445415  351.666524   
1  685.714744  371.518121  95.370776  396.884463  95.370776  357.681934   

          16y         17x         17y  
0  101.121511  414.103575  102.540535  
1  105.747917  417.638743  105.747917  

[2 rows x 36 columns]


# Step 2: Clean and filter incomplete rows 

# Step 4: Create derived features: speed, angle, joint distance 

In [ ]:
# Compute vectors base on implied movement between each pose

def pack_coord(joint_no,pose_no, df):
    joint_str = f"{joint_no:02d}"
    coord = [df[joint_str + "x"][pose_no]]
    coord.append([df[joint_str + "y"]][pose_no])
    return coord

def compute_vect(p1, p2):
    return ([p1[0]-p2[0],p1[1]-p2[1]])



# Step 5: Slice sequences by frame intervals 

In [ ]:
# Not needed for this phase, but possibly useful later for going from video to pose

# Step 6: Assign dance move labels 

# Step 7: Convert pose DataFrame to movement Vectors